# ReAct Agent with Query Engine Tools

## Build Query Engine Tools

In [61]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    ServiceContext,
    load_index_from_storage,
)

from llama_index.tools import QueryEngineTool, ToolMetadata

In [103]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-0613")
# llm = OpenAI(temperature=0, model="gpt-4-0613")
service_context = ServiceContext.from_defaults(llm=llm)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0.3)
)
gpt4_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-4-0613", temperature=0.3))

In [70]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/march")
    march_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/june")
    june_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/sept")
    sept_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [71]:
if not index_loaded:
    # load data
    march_docs = SimpleDirectoryReader(
        input_files=["../data/10q/uber_10q_march_2022.pdf"]
    ).load_data()
    june_docs = SimpleDirectoryReader(
        input_files=["../data/10q/uber_10q_june_2022.pdf"]
    ).load_data()
    sept_docs = SimpleDirectoryReader(
        input_files=["../data/10q/uber_10q_sept_2022.pdf"]
    ).load_data()

    # build index
    march_index = VectorStoreIndex.from_documents(march_docs, service_context=service_context)
    june_index = VectorStoreIndex.from_documents(june_docs, service_context=service_context)
    sept_index = VectorStoreIndex.from_documents(sept_docs, service_context=service_context)

    # persist index
    march_index.storage_context.persist(persist_dir="./storage/march")
    june_index.storage_context.persist(persist_dir="./storage/june")
    sept_index.storage_context.persist(persist_dir="./storage/sept")

In [72]:
march_engine = march_index.as_query_engine(similarity_top_k=3, service_context=service_context)
june_engine = june_index.as_query_engine(similarity_top_k=3, service_context=service_context)
sept_engine = sept_index.as_query_engine(similarity_top_k=3, service_context=service_context)

In [73]:
from llama_index.tools.query_engine import QueryEngineTool


query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=f"Provides information about Uber quarterly financials ending September 2022",
)
query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=f"Provides information about Uber quarterly financials ending June 2022",
)
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=f"Provides information about Uber quarterly financials ending March 2022",
)

query_engine_tools = [query_tool_march, query_tool_june, query_tool_sept]

## Setup ReAct Agent

In [83]:
from llama_index.agent import ReActAgent
from llama_index.llms import OpenAI

In [96]:
# llm = OpenAI(model="gpt-3.5-turbo-0613")
llm = OpenAI(model="gpt-4-0613")
agent = ReActAgent.from_tools(
    query_engine_tools, 
    llm=llm,
    # llm=gpt4,
    callback_manager=callback_manager,
    verbose=True
)

In [ ]:
response = agent.chat("Analyze Uber revenue growth over the last few quarters")
print(str(response))

In [ ]:
print(str(response))

In [97]:
# chatgpt-3.5 doesn't give the right response, but gpt-4 does
response = agent.chat("Can you tell me about the risk factors in the quarter with the highest revenue growth?")
print(str(response))

Thought: To answer this question, I need to find the quarter with the highest revenue growth first. I will use the tools march_2022, june_2022, and sept_2022 to get the revenue data for each quarter.
Action: march_2022
Action Input: {'input': 'revenue'}
Observation: We generate substantially all of our revenue from fees paid by Drivers and Merchants for use of our platform. We act as an agent in these transactions by connecting consumers to Drivers and Merchants to facilitate a Trip, meal, or grocery delivery service. In certain markets, where we are responsible for mobility services, we present revenue from end-users on a gross basis. Our revenue is net of Driver and Merchant earnings and Driver incentives. We recognize revenue when a trip is complete.
Thought: The response from the tool doesn't provide a specific revenue figure for the March 2022 quarter. I'll need to ask for the revenue figure specifically.
Action: march_2022
Action Input: {'input': 'revenue figure'}
Observation: Th

In [59]:
finetuning_handler.save_finetuning_events('tmp.jsonl')

Wrote 5 examples to tmp.jsonl


In [ ]:
import json

for line in open('tmp.jsonl', 'r'):
    display(json.loads(line))

## Generate Training/Eval Questions

Generate a synthetic dataset of questions to ask. To do this, we generate an initial set of questions over a "base" document (the March 2022 10Q), and then we use an LLM to generate variations of that question that can apply across multiple quarters. This allows us to more deeply stress-test the LLM reasoning capabilities.


In [104]:
from llama_index.evaluation import DatasetGenerator

In [105]:
base_question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context from the Uber March 10Q filing, formulate "
    "a single question that captures an important fact from the context. "
    "context. Restrict the question to the context information provided."
)

dataset_generator = DatasetGenerator.from_documents(
    march_docs,
    question_gen_query=base_question_gen_query,
    service_context=gpt_35_context,
)

In [106]:
questions = dataset_generator.generate_questions_from_nodes(num=20)

In [107]:
questions

["What is the address of Uber Technologies, Inc.'s principal executive offices?",
 "What are the financial statements included in Uber's March 10Q filing?",
 'What are some of the factors that Uber identifies as potential impacts on its business operations and financial performance?',
 "What is the company's stance on updating forward-looking statements in their Quarterly Report on Form 10-Q?",
 "What is the total amount of cash and cash equivalents as of March 31, 2022, according to Uber's March 10Q filing?",
 'What was the net loss attributable to Uber Technologies, Inc. for the three months ended March 31, 2022?',
 'What was the comprehensive income (loss) attributable to Uber Technologies, Inc. for the three months ended March 31, 2022?',
 'What was the balance of non-redeemable non-controlling interests as of March 31, 2021, according to the Uber March 10Q filing?',
 'What was the net income (loss) for Uber Technologies, Inc. for the period ending March 31, 2022?',
 'What was the 

In [125]:
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate


vary_question_tmpl = """\
You are a financial assistant. Given a question over a 2023 Uber 10Q filing, your goal
is to generate up to {num_vary} variations of that question that might span multiple 10Q's.

This can include compare/contrasting different 10Qs, replacing the current quarter with
another quarter, or generating questions that can only be answered over multiple quarters (be creative!)

You are given a valid set of 10Q filings. Please only generate question variations that can be
answered in that set.

For example:
Base Question: What was the free cash flow of Uber in March 2023?
Valid 10Qs: [March 2023, June 2023, September 2023]
Question Variations:
What was the free cash flow of Uber in June 2023?
Can you compare/contrast the free cash flow of Uber in June/September 2023 and offer explanations for the change?
Did the free cash flow of Uber increase of decrease in 2023?

Now let's give it a shot! 

Base Question: {base_question}
Valid 10Qs: {valid_10qs}
Question Variations:
"""

def gen_question_variations(base_questions, num_vary=3):
    """Generate question variations."""

    VALID_10Q_STR = "[March 2022, June 2022, September 2022]"
    
    llm = OpenAI(model="gpt-4")
    prompt_tmpl = PromptTemplate(vary_question_tmpl)

    new_questions = []
    for idx, question in enumerate(base_questions):
        new_questions.append(question)
        response = llm.complete(prompt_tmpl.format(num_vary=num_vary, base_question=question, valid_10qs=VALID_10Q_STR))
        # parse into newlines
        raw_lines = str(response).split("\n")
        cur_new_questions = [l for l in raw_lines if l != ""]
        print(f'[{idx}] Original Question: {question}')
        print(f'[{idx}] Generated Question Variations: {cur_new_questions}')
        new_questions.extend(cur_new_questions)

    return new_questions

def save_questions(questions, path):
    with open(path, "w") as f:
        for question in questions:
            f.write(question + "\n")


def load_questions(path):
    questions = []
    with open(path, "r") as f:
        for line in f:
            questions.append(line.strip())
    return questions

In [120]:
new_questions = gen_question_variations(questions)

Original Question: What is the address of Uber Technologies, Inc.'s principal executive offices?
Generated Question Variations: ["Has the address of Uber Technologies, Inc.'s principal executive offices changed between March and September 2022?", "What was the address of Uber Technologies, Inc.'s principal executive offices in June 2022?", "Can you track the changes in the address of Uber Technologies, Inc.'s principal executive offices throughout 2022?"]
Original Question: What are the financial statements included in Uber's March 10Q filing?
Generated Question Variations: ["What are the financial statements included in Uber's June 2022 10Q filing?", "Can you compare and contrast the financial statements included in Uber's March and June 2022 10Q filings?", "How have the financial statements included in Uber's 10Q filings evolved over the course of 2022?"]
Original Question: What are some of the factors that Uber identifies as potential impacts on its business operations and financial

In [126]:
len(new_questions)

80

In [127]:
train_questions, eval_questions = new_questions[:60], new_questions[60:]

In [131]:
save_questions(train_questions, "train_questions_10q.txt")
save_questions(eval_questions, "eval_questions_10q.txt")

In [132]:
train_questions = load_questions("train_questions_10q.txt")
eval_questions = load_questions("eval_questions_10q.txt")

## Use GPT-4 to Log Input/Output Pairs

In [139]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager
from llama_index.agent import ReActAgent

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [140]:
llm = OpenAI(model="gpt-4-0613")
agent = ReActAgent.from_tools(
    query_engine_tools, 
    llm=llm,
    callback_manager=callback_manager,
    verbose=True
)

In [ ]:
for idx, question in enumerate(train_questions):
    print(f"[{idx}] Question: {question}")
    response = agent.query(question)
    print(f"[{idx}] Agent Response: {str(response)}")

[0] Question: What is the address of Uber Technologies, Inc.'s principal executive offices?
Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'principal executive offices address'}
Observation: I'm sorry, but I cannot provide the principal executive offices address based on the given context information.
Thought: The first tool didn't provide the needed information. I should try another tool.
Action: june_2022
Action Input: {'input': 'principal executive offices address'}
Observation: Sorry, but I can't help with that request.
Thought: The second tool also didn't provide the needed information. I should try the last tool.
Action: sept_2022
Action Input: {'input': 'principal executive offices address'}
Observation: Sorry, but I can't help with that request.
Response: I'm sorry, but I don't have the specific information about the address of Uber Technologies, Inc.'s principal executive offices.
[0] Agent Response: I'm sorry, but I do